# Load libraries

In [1]:
import numpy as np
import tensorflow as tf

c:\users\silver\appdata\local\programs\python\python35\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Load MNIST

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


# Define Model (Estimator)

In [4]:
def model_fn(features, labels, mode):
    # Define Model parameters
    W = tf.Variable(tf.zeros([784, 10]))
    b = tf.Variable(tf.zeros([10]))
    
    # Define Model structure  y = f(x)
    y = tf.matmul(features['x'], W) + b
    
    # Define Model Loss
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=y))
    
    # Define Optimizer and step
    global_step = tf.train.get_global_step()
    optimizer = tf.train.GradientDescentOptimizer(0.5)
    train = tf.group(optimizer.minimize(cross_entropy),
                     tf.assign_add(global_step, 1))
    
    # Define other evaluation metrics 
    eval_metric_ops = {
      "accuracy": tf.metrics.accuracy(tf.argmax(labels,1), tf.argmax(y,1))
    }
    return tf.estimator.EstimatorSpec(
              mode=mode,
              predictions=y,
              loss=cross_entropy,
              train_op=train,
              eval_metric_ops=eval_metric_ops)

x_train = mnist.train.images
y_train = mnist.train.labels.astype("float32")
x_eval = mnist.test.images
y_eval = mnist.test.labels.astype("float32")

estimator = tf.estimator.Estimator(model_fn=model_fn)
# define the training & evaluation settings
input_fn = tf.estimator.inputs.numpy_input_fn(
                {"x": x_train},
                y_train,
                batch_size=128,
                num_epochs=None,
                shuffle=True)
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                {"x": x_train},
                y_train,
                batch_size=128,
                num_epochs=1,
                shuffle=False)
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
                {"x": x_eval},
                y_eval,
                batch_size=128,
                num_epochs=1,
                shuffle=False)

# Training & Evaluation

In [5]:
# Training
estimator.train(input_fn=input_fn, steps=1000)

# evaluation
train_metrics = estimator.evaluate(input_fn=train_input_fn)["accuracy"]
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)["accuracy"]
print("\nTrain Accuracy: {0:f}\n".format(train_metrics))
print("\nTest Accuracy: {0:f}\n".format(eval_metrics))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_num_worker_replicas': 1, '_task_id': 0, '_is_chief': True, '_save_summary_steps': 100, '_master': '', '_tf_random_seed': None, '_service': None, '_task_type': 'worker', '_keep_checkpoint_every_n_hours': 10000, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000236F2591080>, '_session_config': None, '_save_checkpoints_steps': None, '_model_dir': 'C:\\Users\\silver\\AppData\\Local\\Temp\\tmpyhvmldwv', '_num_ps_replicas': 0, '_save_checkpoints_secs': 600, '_log_step_count_steps': 100, '_keep_checkpoint_max': 5}
INFO:tensorflow:Create CheckpointSaverHook.


InternalError: Blas GEMM launch failed : a.shape=(128, 784), b.shape=(784, 10), m=128, n=10, k=784
	 [[Node: MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](random_shuffle_queue_DequeueMany/_49, Variable/read, ^add)]]
	 [[Node: Mean/_55 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_83_Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'MatMul', defined at:
  File "c:\users\silver\appdata\local\programs\python\python35\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "c:\users\silver\appdata\local\programs\python\python35\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "c:\users\silver\appdata\local\programs\python\python35\lib\site-packages\ipykernel_launche